In [30]:
#install.packages("stringr")
#install.packages("ggthemes")
#install.packages("devtools")
#devtools::install_github("tidyverse/readr")
#install.packages("dplyr")
#install.packages("maps")
#install.packages("readr")
install.packages("ggplot2")

Updating HTML index of packages in '.Library'
Making 'packages.html' ... done


In [31]:
library(dplyr)
library(stringr)
library(maps)
library(ggplot2)

suppressPackageStartupMessages(library(tidyverse))
library(stringr)
library(ggthemes)
library(dplyr)
suppressPackageStartupMessages(library(maps))

In [9]:
charges <- read.csv('/Users/chriskuo/downloads/Inpatient_Payment.csv')

In [10]:
print("Data size in RAM:")
print(object.size(charges), units = 'Mb')

print("Data.frame dimensions:")
print(dim(charges))

head(charges)

[1] "Data size in RAM:"
36.1 Mb
[1] "Data.frame dimensions:"
[1] 163065     12


DRG.Definition,Provider.Id,Provider.Name,Provider.Street.Address,Provider.City,Provider.State,Provider.Zip.Code,Hospital.Referral.Region.Description,Total.Discharges,Average.Covered.Charges,Average.Total.Payments,Average.Medicare.Payments
039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,10001,SOUTHEAST ALABAMA MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,AL - Dothan,91,$32963.07,$5777.24,$4763.73
039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,10005,MARSHALL MEDICAL CENTER SOUTH,2505 U S HIGHWAY 431 NORTH,BOAZ,AL,35957,AL - Birmingham,14,$15131.85,$5787.57,$4976.71
039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,10006,ELIZA COFFEE MEMORIAL HOSPITAL,205 MARENGO STREET,FLORENCE,AL,35631,AL - Birmingham,24,$37560.37,$5434.95,$4453.79
039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,10011,ST VINCENT'S EAST,50 MEDICAL PARK EAST DRIVE,BIRMINGHAM,AL,35235,AL - Birmingham,25,$13998.28,$5417.56,$4129.16
039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,10016,SHELBY BAPTIST MEDICAL CENTER,1000 FIRST STREET NORTH,ALABASTER,AL,35007,AL - Birmingham,18,$31633.27,$5658.33,$4851.44
039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,10023,BAPTIST MEDICAL CENTER SOUTH,2105 EAST SOUTH BOULEVARD,MONTGOMERY,AL,36116,AL - Montgomery,67,$16920.79,$6653.80,$5374.14


In [14]:
names(charges) <- c('drg_def', 'prov_id', 'prov_name', 'prov_address', 'prov_city', 'prov_state', 
                   'prov_zip', 'referral_reg', 'total_discharges', 'mean_covered_charges',
                   'mean_total_payments', 'mean_medicare_payments')

##### Note: 
All the variants of summarise, mutate and transmute() make the tasks easy to apply to selected variables.
* summarise_all(), summarise_if(), summarise_at(), summarize_all(), summarize_if(), summarize_at(), 
* mutate_all(), mutate_if(), mutate_at(), 
* transmute_all(), transmute_if(), transmute_at()

There are three variants.
* _all(): apples every variable.
* _if(): apples variables selected with a predicate function.
* _at(): applies variables selected with a character vector or vars().

In [15]:
remove_dollar <- function(x) {
    as.numeric(str_replace(x, '\\$', ''))
}

charges <- charges %>% 
    mutate_at(vars(mean_covered_charges, mean_total_payments, mean_medicare_payments), 
              remove_dollar)
head(charges)

drg_def,prov_id,prov_name,prov_address,prov_city,prov_state,prov_zip,referral_reg,total_discharges,mean_covered_charges,mean_total_payments,mean_medicare_payments
039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,10001,SOUTHEAST ALABAMA MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,AL - Dothan,91,32963.07,5777.24,4763.73
039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,10005,MARSHALL MEDICAL CENTER SOUTH,2505 U S HIGHWAY 431 NORTH,BOAZ,AL,35957,AL - Birmingham,14,15131.85,5787.57,4976.71
039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,10006,ELIZA COFFEE MEMORIAL HOSPITAL,205 MARENGO STREET,FLORENCE,AL,35631,AL - Birmingham,24,37560.37,5434.95,4453.79
039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,10011,ST VINCENT'S EAST,50 MEDICAL PARK EAST DRIVE,BIRMINGHAM,AL,35235,AL - Birmingham,25,13998.28,5417.56,4129.16
039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,10016,SHELBY BAPTIST MEDICAL CENTER,1000 FIRST STREET NORTH,ALABASTER,AL,35007,AL - Birmingham,18,31633.27,5658.33,4851.44
039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,10023,BAPTIST MEDICAL CENTER SOUTH,2105 EAST SOUTH BOULEVARD,MONTGOMERY,AL,36116,AL - Montgomery,67,16920.79,6653.80,5374.14


## Exploratory Data Analysis (EDA)

#### DRG: Diagnosis Related Group
A classification system of diagnosis used by Medicare & Medicaid that groups patients according to diagnosis, type of treatment, age, and other relevant criteria, regardless the actual cost of care for the individual. DRGs are assigned based on ICD (International Classification of Diseases) diagnoses, procedures, age, sex, discharge status, and the presence of complications or comorbidities. DRGs have been used in the US since 1982 to determine how much Medicare pays the hospital for each "product", since patients within each category are clinically similar and are expected to use the same level of hospital resources. DRGs may be further grouped into Major Diagnostic Categories (MDCs). DRGs are also standard practice for establishing reimbursements for other Medicare related reimbursements such as to home healthcare providers.[citation needed]

#### What are the top ten most common procedures.

In [17]:
charges %>%
    group_by(drg_def) %>%
    summarize(n = n()) %>%
    arrange(-n) %>% # Use arrange to sort in descending order
    head(10)

drg_def,n
194 - SIMPLE PNEUMONIA & PLEURISY W CC,3023
690 - KIDNEY & URINARY TRACT INFECTIONS W/O MCC,2989
292 - HEART FAILURE & SHOCK W CC,2953
"392 - ESOPHAGITIS, GASTROENT & MISC DIGEST DISORDERS W/O MCC",2950
"641 - MISC DISORDERS OF NUTRITION,METABOLISM,FLUIDS/ELECTROLYTES W/O MCC",2899
871 - SEPTICEMIA OR SEVERE SEPSIS W/O MV 96+ HOURS W MCC,2812
603 - CELLULITIS W/O MCC,2807
470 - MAJOR JOINT REPLACEMENT OR REATTACHMENT OF LOWER EXTREMITY W/O MCC,2750
191 - CHRONIC OBSTRUCTIVE PULMONARY DISEASE W CC,2720
190 - CHRONIC OBSTRUCTIVE PULMONARY DISEASE W MCC,2713


In [34]:
# https://www.cms.gov/icd10manual/fullcode_cms/P0002.html
procDF <- charges %>% # aggregate  procedures from all hospitals
  group_by(drg_def) %>% 
  summarise(procSum = sum(total_discharges))

#procDF <- data.frame(procDF)
#procDF <- procDF[order(-procDF$procSum),] # sort by #procedures

#x <- list(  title = "procedure type")
#y <- list(  title = "procedure count")

In [35]:
head(procDF)

drg_def,procSum
039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,33606
057 - DEGENERATIVE NERVOUS SYSTEM DISORDERS W/O MCC,30212
064 - INTRACRANIAL HEMORRHAGE OR CEREBRAL INFARCTION W MCC,62093
065 - INTRACRANIAL HEMORRHAGE OR CEREBRAL INFARCTION W CC,106414
066 - INTRACRANIAL HEMORRHAGE OR CEREBRAL INFARCTION W/O CC/MCC,55849
069 - TRANSIENT ISCHEMIA,79590


In [39]:
ggplot(data=procDF, aes(x=drg_def, y=procSum))

# geom_bar is designed to make it easy to create bar charts that show counts
g <- ggplot(procDF, aes(drg_def,procSum))
g + geom_bar(stat = "identity", aes(fill = type))


ERROR: Error in dev.off(): QuartzBitmap_Output - unable to open file '/var/folders/jw/wyhtzlf94zbgtpf9g_n5tryr0000gn/T//Rtmps8NJjv/file301b5df5a074.png'


plot without title

ERROR: Error in dev.off(): QuartzBitmap_Output - unable to open file '/var/folders/jw/wyhtzlf94zbgtpf9g_n5tryr0000gn/T//Rtmps8NJjv/file301b10fe2034.png'


plot without title

In [44]:
# Simple Bar Plot 
counts <- table(procDF$procSum)
barplot(counts, main="DRG", 
  	xlab="Total")

ERROR: Error in dev.off(): QuartzBitmap_Output - unable to open file '/var/folders/jw/wyhtzlf94zbgtpf9g_n5tryr0000gn/T//Rtmps8NJjv/file301b1e6b2635.png'


Plot with title “DRG”

In [43]:
setwd("/Users/chriskuo/downloads")

In [32]:
# Add codes for DC and Puerto Rico to the default state lists
state.abb <- append(state.abb, c("DC"))
state.name <- append(state.name, c("District of Columbia"))

# Map the state abbreviations to state names so we can join with the map data
charges$region <- map_chr(charges$prov_state, function(x) { tolower(state.name[grep(x, state.abb)]) })

# Get the us state map data
state_map <- map_data('state')

ERROR: Error: Result 160 is not a length 1 atomic vector


In [7]:
state_map <- map_data('state')

ERROR: Error in map_data("state"): could not find function "map_data"


In [30]:
library(purrr)

In [38]:
states <- map_data("state")

ERROR: Error: ggplot2 doesn't know how to deal with data of class list
